In [1]:
# !pip install transformers

In [2]:
from transformers import pipeline, AutoTokenizer
import torch
import time

pipe = pipeline("text-generation",
                model="bigscience/bloom-560m",
                device=0,
                torch_dtype=torch.float16)

In [17]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
start_text = "Testing BLOOM-560M without DeepSpeed"
tokens_start_text = len(tokenizer(start_text, return_tensors="pt").input_ids[0])

In [18]:
new_tokens = 500

t0 = time.time()
result = pipe(start_text, max_new_tokens=new_tokens)
t1 = time.time()

In [19]:
gen_text = result[0]['generated_text']
tokens_gen_text = len(tokenizer(gen_text, return_tensors="pt").input_ids[0])
total_new_tokens_generated = tokens_gen_text - tokens_start_text
throughput = (total_new_tokens_generated) / (t1 - t0)

In [22]:
print(f"""Tokens generated: {total_new_tokens_generated}
Time: {t1 - t0:.1f} seconds
Tokens per second: {throughput:.1f}
Latency: {1000 / throughput:.1f} ms""")

Tokens generated: 500
Time: 9.3 seconds
Tokens per second: 54.0
Latency: 18.5 ms
